In [150]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows',500)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb

/home/alexzeng/anaconda3/envs/sepsis-project/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [139]:
df_admi = pd.read_csv('./dataset/ADMISSIONS.csv')
df_diag = pd.read_csv('./dataset/DIAGNOSES_ICD.csv')
df_los = pd.read_csv('./dataset/ICUSTAYS.csv')
df_patient = pd.read_csv('./dataset/PATIENTS.csv')

df_admi = df_admi.dropna()
df_admi = df_admi.drop_duplicates(subset=['SUBJECT_ID'],keep = 'first')
df_admi.drop([
    "ROW_ID",
    "ADMITTIME",
    "DISCHTIME",
    "DEATHTIME",
    "EDREGTIME",
    "EDOUTTIME",
    "HOSPITAL_EXPIRE_FLAG",
    "HAS_CHARTEVENTS_DATA",
    "DIAGNOSIS",
    "LANGUAGE",
    "HADM_ID"
], axis=1, inplace=True)

# df_diag = df_diag.drop_duplicates(subset=['SUBJECT_ID','HADM_ID'],keep = 'first')
df_diag = df_diag.dropna()
df_diag.drop([
    "ROW_ID",
    "SEQ_NUM",
    "HADM_ID"
],axis = 1, inplace = True)
# df_diag = df_diag.loc[df_diag["ICD9_CODE"].str.contains('99591')]
df_diag.loc[df_diag["ICD9_CODE"].str.contains('99591'),"ICD9_CODE"] = '\+1'
df_diag.loc[~df_diag["ICD9_CODE"].str.contains('\+1'),"ICD9_CODE"]= '\-1' 
# df_diag = df_diag.loc[df_diag["ICD9_CODE"] != "99591","ICD9_CODE"] = '0'
# df_diag = df_diag.loc[df_diag["ICD9_CODE"].str.contains('\-1')]
def label_sepsis (e):
    if e in '\+1':
        return 1
    else:
        return 0
df_diag['is_sepsis'] = df_diag['ICD9_CODE'].apply(lambda e :label_sepsis(e))
df_diag.drop([
    "ICD9_CODE",
],axis = 1, inplace = True)
df_diag = df_diag.sort_values('is_sepsis')
df_diag = df_diag.drop_duplicates(subset = 'SUBJECT_ID', keep = "last")
# df_diag = df_diag.loc[df_diag['is_sepsis'] == 0]

# df_los = df_los.drop_duplicates(subset=['SUBJECT_ID'],keep = 'last')
df_los = df_los.dropna()
df_los = df_los.groupby('SUBJECT_ID').mean().reset_index()
df_los.drop([
    "ROW_ID",
    "ICUSTAY_ID",
    "FIRST_WARDID",
    "LAST_WARDID",
    "HADM_ID"
],axis = 1, inplace = True)
df_los.LOS = df_los.LOS.astype(int)
df_los = df_los.loc[df_los['LOS']>=1]

# df_patient = df_patient.drop_duplicates(subset=['SUBJECT_ID','HADM_ID'],keep = 'first')
df_patient = df_patient.dropna()
df_patient['DOB'] = pd.to_datetime(df_patient['DOB']).dt.date
df_patient['DOD'] = pd.to_datetime(df_patient['DOD']).dt.date
df_patient['age'] = df_patient.apply(lambda e:(e['DOD'] - e['DOB']).days/365,axis = 1)
df_patient.age = df_patient.age.astype(int)
df_patient.drop([
    "ROW_ID",
    "EXPIRE_FLAG",
    "DOD_SSN",
    "DOD_HOSP",
    "DOD",
    "DOB"
],axis = 1, inplace = True)
df_patient = df_patient.loc[df_patient['age']<=100]

# df_admi
# df_diag
# df_los
# df_patient

df_merge = pd.merge(df_admi, df_diag, on="SUBJECT_ID")
df_merge = pd.merge(df_merge,df_los, on="SUBJECT_ID")
df_merge = pd.merge(df_merge,df_patient, on="SUBJECT_ID")
df_merge.drop([
    "SUBJECT_ID",
    "ADMISSION_TYPE",
    "DISCHARGE_LOCATION"
],axis = 1, inplace = True)
df_merge.dtypes
# df_merge['INSURANCE'].unique()

ADMISSION_LOCATION    object
INSURANCE             object
RELIGION              object
MARITAL_STATUS        object
ETHNICITY             object
is_sepsis              int64
LOS                    int64
GENDER                object
age                    int64
dtype: object

In [140]:
# prediction
X = df_merge.drop('is_sepsis',axis = 1).copy()
X.head()

,ADMISSION_LOCATION,INSURANCE,RELIGION,MARITAL_STATUS,ETHNICITY,LOS,GENDER,age
0,EMERGENCY ROOM ADMIT,Medicaid,NOT SPECIFIED,SINGLE,BLACK/AFRICAN AMERICAN,2,F,25
1,EMERGENCY ROOM ADMIT,Medicare,JEWISH,SINGLE,WHITE,2,M,63
2,CLINIC REFERRAL/PREMATURE,Private,PROTESTANT QUAKER,MARRIED,WHITE,6,M,50
3,EMERGENCY ROOM ADMIT,Private,CATHOLIC,MARRIED,WHITE,3,M,56
4,CLINIC REFERRAL/PREMATURE,Private,CATHOLIC,MARRIED,WHITE,3,M,56


In [141]:
y = df_merge['is_sepsis'].copy()
y.head()

0    0
1    1
2    0
3    0
4    0
Name: is_sepsis, dtype: int64

In [145]:
#one hot encoding
X_encoded = pd.get_dummies(X, columns=[
    'ADMISSION_LOCATION',
    'INSURANCE',
    'RELIGION',
    'MARITAL_STATUS',
    'ETHNICITY',
    'GENDER'
])
X_encoded.head()

,LOS,age,ADMISSION_LOCATION_CLINIC REFERRAL/PREMATURE,ADMISSION_LOCATION_EMERGENCY ROOM ADMIT,ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI,ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM,ADMISSION_LOCATION_TRANSFER FROM OTHER HEALT,ADMISSION_LOCATION_TRANSFER FROM SKILLED NUR,INSURANCE_Government,INSURANCE_Medicaid,...,ETHNICITY_PATIENT DECLINED TO ANSWER,ETHNICITY_PORTUGUESE,ETHNICITY_UNABLE TO OBTAIN,ETHNICITY_UNKNOWN/NOT SPECIFIED,ETHNICITY_WHITE,ETHNICITY_WHITE - EASTERN EUROPEAN,ETHNICITY_WHITE - OTHER EUROPEAN,ETHNICITY_WHITE - RUSSIAN,GENDER_F,GENDER_M
0,2,25,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,2,63,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
2,6,50,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,3,56,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
4,3,56,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [146]:
# only .07 patient got sepsis
# when spliting the training/testing dataset, preserve the precentage
sum(y)/len(y)

0.0761994355597366

In [149]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state = 42, stratify=y)
sum(y_train)/len(y_train)
sum(y_test)/len(y_test)

0.07518796992481203

In [ ]:
clf_xgb = xgb.X